# Предсказание вероятности клика по рекламному объявлению

**Краткое описание**

Требуется получить на выходе прогноз с высокой точностью вероятности клика по рекламному объявлению.
<br>На предикт ML модели выдается выборка объявлений доступных в текущий момент к показу, модель выдает по ним вероятность клика.

**Задача**

- Выбор модели для получения предсказаний.
- Обосновавние выбора

**План выполнения**
- Знакомство с данными
- Исследовательский анализ данных
- Предобработка данных (при необходимости)
- Разработка дополнительных признаков (при необходимости)
- Построение baseline
- Обучение нескольких моделей
- Выбор лучшей модели
- Проверка лучшей модели на отложенной выборке
- Проверка важности признаков
- Заключение о проделанном исследовании

**Описание данных**
- `region_id` - id региона показа
- `city_id` - id города
- `tags_cont` - значение контекстного таргетинга
- `tags_bhv` - значение таргетинга по поведению
- `rubrica` - тематика объявления
- `rate` - ставка в рублях объявления
- `ctr_sort` - значение ctr объявления
- `rv_perc` - процент видимости объявления
- `slider` - тип рекламного блока
- `position_type` - значение позиции рекламного блока на странице сайта

**Метрики для оценки качества модели**
- основная метрика - `ROC AUC`
- дополнительная - `F1`

## Установка зависимостей

In [17]:
# !pip install pandas
# !pip install seaborn
# !pip freeze > requirements.txt

In [7]:
import os
import pandas as pd
import seaborn as sns

Объявление глобальных переменных

In [8]:
RANDOM_STATE = 42
DATA_DIR = 'data'

pd.options.display.max_columns = 100
pd.options.display.max_rows = 64
pd.options.mode.copy_on_write = True

sns.set_style("white")
sns.set_theme(style="whitegrid")

%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

## Знакомство с данными

In [41]:
data = pd.read_csv(os.path.join(DATA_DIR, 'dataset.csv'))
data_y23 = pd.read_csv(os.path.join(DATA_DIR, 'dataset_2023_10_30.csv'))

In [36]:
def data_review(dataset):
    '''
    на вход принимает датафрейм,
    выводит общую обзорную информацию
    '''
    print('Общая информация о наборе данных:')
    dataset.info()
    print()
    print('Первые пять строк набора данных:')
    display(dataset.head())
    print()
    print(f"количество полных дубликатов строк: {dataset.duplicated().sum()}")
    print()
    print(f"""количество пропущенных значений: 
    {dataset.isna().sum()}""")
    print()
    print('Вывод количества уникальных записей в каждом числовом признаке:')
    for column in dataset.select_dtypes(include=['int', 'float']).columns:
        unique_values = dataset[column].nunique()
        print(f"Количество уникальных записей в признаке '{column}': {unique_values}")

### dataset

In [42]:
data_review(data)

Общая информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803526 entries, 0 to 803525
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   region_id      803526 non-null  int64  
 1   city_id        803526 non-null  int64  
 2   tags_cont      803526 non-null  float64
 3   tags_bhv       803526 non-null  float64
 4   rubrica        803526 non-null  float64
 5   rate           803526 non-null  float64
 6   ctr_sort       803526 non-null  float64
 7   rv_perc        803526 non-null  float64
 8   slider         803526 non-null  int64  
 9   position_type  803526 non-null  int64  
 10  target         803526 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 67.4 MB

Первые пять строк набора данных:


,region_id,city_id,tags_cont,tags_bhv,rubrica,rate,ctr_sort,rv_perc,slider,position_type,target
0,38,590,0.0,0.500,1.812,1.7,1.3072,66.41,1,0,1
1,53,577,0.0,0.000,0.000,4.8,0.6568,46.79,1,0,1
2,48,0,0.0,0.000,1.950,1.7,1.3072,66.41,1,0,1
3,20,596,0.0,1.103,0.000,1.7,1.3072,94.44,0,0,1
4,290,0,0.0,0.750,0.000,1.0,1.1369,77.69,0,0,1



количество полных дубликатов строк: 255574

количество пропущенных значений: 
    region_id        0
city_id          0
tags_cont        0
tags_bhv         0
rubrica          0
rate             0
ctr_sort         0
rv_perc          0
slider           0
position_type    0
target           0
dtype: int64

Вывод количества уникальных записей в каждом числовом признаке:
Количество уникальных записей в признаке 'region_id': 228
Количество уникальных записей в признаке 'city_id': 174
Количество уникальных записей в признаке 'tags_cont': 1527
Количество уникальных записей в признаке 'tags_bhv': 4687
Количество уникальных записей в признаке 'rubrica': 3153
Количество уникальных записей в признаке 'rate': 64
Количество уникальных записей в признаке 'ctr_sort': 3085
Количество уникальных записей в признаке 'rv_perc': 5238
Количество уникальных записей в признаке 'slider': 2
Количество уникальных записей в признаке 'position_type': 26
Количество уникальных записей в признаке 'target': 2


### dataset_23

In [43]:
data_review(data_y23)

Общая информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648184 entries, 0 to 648183
Data columns (total 11 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   region_id      648184 non-null  int64  
 1   city_id        648184 non-null  int64  
 2   tags_cont      648184 non-null  float64
 3   tags_bhv       648184 non-null  float64
 4   rubrica        648184 non-null  float64
 5   rate           648184 non-null  float64
 6   ctr_sort       648184 non-null  float64
 7   rv_perc        648184 non-null  float64
 8   slider         648184 non-null  int64  
 9   position_type  648184 non-null  int64  
 10  target         648184 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 54.4 MB

Первые пять строк набора данных:


,region_id,city_id,tags_cont,tags_bhv,rubrica,rate,ctr_sort,rv_perc,slider,position_type,target
0,38,617,0.0,0.000,0.000,26.25,0.6608,89.97,0,0,1
1,284,0,0.0,0.000,1.750,1.50,1.8803,27.31,0,0,1
2,157,0,0.0,2.576,1.488,0.70,1.3899,78.42,1,0,1
3,228,0,0.0,0.000,0.000,0.75,1.6981,7.75,0,21,1
4,51,579,0.0,0.000,0.000,1.60,1.3530,77.69,0,0,1



количество полных дубликатов строк: 205732

количество пропущенных значений: 
    region_id        0
city_id          0
tags_cont        0
tags_bhv         0
rubrica          0
rate             0
ctr_sort         0
rv_perc          0
slider           0
position_type    0
target           0
dtype: int64

Вывод количества уникальных записей в каждом числовом признаке:
Количество уникальных записей в признаке 'region_id': 226
Количество уникальных записей в признаке 'city_id': 183
Количество уникальных записей в признаке 'tags_cont': 7054
Количество уникальных записей в признаке 'tags_bhv': 8622
Количество уникальных записей в признаке 'rubrica': 2732
Количество уникальных записей в признаке 'rate': 76
Количество уникальных записей в признаке 'ctr_sort': 3364
Количество уникальных записей в признаке 'rv_perc': 5673
Количество уникальных записей в признаке 'slider': 2
Количество уникальных записей в признаке 'position_type': 24
Количество уникальных записей в признаке 'target': 2


### Выводы
- оба набора данных представлены исключительно числовыми данными;
- в обоих наборах примерно 30% строк являются повторами;
- в наборах отсутствуют идентификаторы рекламных объявлений;
- ввиду схожести наборов и отсутствия идентификаторов принимаю решение их объединить

### common_df

In [47]:
df = pd.concat([data, data_y23], axis=0, ignore_index=True)

In [48]:
data_review(df)

Общая информация о наборе данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1451710 entries, 0 to 1451709
Data columns (total 11 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   region_id      1451710 non-null  int64  
 1   city_id        1451710 non-null  int64  
 2   tags_cont      1451710 non-null  float64
 3   tags_bhv       1451710 non-null  float64
 4   rubrica        1451710 non-null  float64
 5   rate           1451710 non-null  float64
 6   ctr_sort       1451710 non-null  float64
 7   rv_perc        1451710 non-null  float64
 8   slider         1451710 non-null  int64  
 9   position_type  1451710 non-null  int64  
 10  target         1451710 non-null  int64  
dtypes: float64(6), int64(5)
memory usage: 121.8 MB

Первые пять строк набора данных:


,region_id,city_id,tags_cont,tags_bhv,rubrica,rate,ctr_sort,rv_perc,slider,position_type,target
0,38,590,0.0,0.500,1.812,1.7,1.3072,66.41,1,0,1
1,53,577,0.0,0.000,0.000,4.8,0.6568,46.79,1,0,1
2,48,0,0.0,0.000,1.950,1.7,1.3072,66.41,1,0,1
3,20,596,0.0,1.103,0.000,1.7,1.3072,94.44,0,0,1
4,290,0,0.0,0.750,0.000,1.0,1.1369,77.69,0,0,1



количество полных дубликатов строк: 461322

количество пропущенных значений: 
    region_id        0
city_id          0
tags_cont        0
tags_bhv         0
rubrica          0
rate             0
ctr_sort         0
rv_perc          0
slider           0
position_type    0
target           0
dtype: int64

Вывод количества уникальных записей в каждом числовом признаке:
Количество уникальных записей в признаке 'region_id': 245
Количество уникальных записей в признаке 'city_id': 183
Количество уникальных записей в признаке 'tags_cont': 7301
Количество уникальных записей в признаке 'tags_bhv': 8813
Количество уникальных записей в признаке 'rubrica': 3843
Количество уникальных записей в признаке 'rate': 95
Количество уникальных записей в признаке 'ctr_sort': 5584
Количество уникальных записей в признаке 'rv_perc': 7679
Количество уникальных записей в признаке 'slider': 2
Количество уникальных записей в признаке 'position_type': 26
Количество уникальных записей в признаке 'target': 2


## Исследовательский анализ данных